# Best Practices with OpenMC

In this tutorial, we'll learn about how to use OpenMC efficiently and correctly.

## Lattices vs. Flat Geometries

To obtain good tracking performance (particles/s that OpenMC can simulate), you should think carefully about how the geometry is constructed. In general, universes and lattices will perform better than "flat" universes, because when a particle moves into an adjacent cell, a lattice immediately has a "rule" for finding the identity of the adjacent cell.

In [ ]:
import openmc

model = openmc.Model()
model.settings.particles = 10000
model.settings.inactive = 50
model.settings.batches = 100

In [ ]:
mat1 = openmc.Material()
mat1.add_element('U', 1.0)
mat1.set_density('g/cm3', 11.0)

mat2 = openmc.Material()
mat2.add_nuclide('Pu239', 1.0)
mat2.set_density('g/cm3', 11.0)

zcyl = openmc.ZCylinder(r=0.4)
cyl = openmc.Cell(fill=mat1, region=-zcyl)
ocyl = openmc.Cell(fill=mat2, region=+zcyl)

univ = openmc.Universe(cells=[ocyl, cyl])

Let's build a geometry containing $N^2$ squares all filled with this material, in one of two different ways:

- a NxN rectilinear lattice (a "nested" geometry)
- building the x-y planes and filling our universe into each slot (a "flat" geometry)

In [ ]:
N = 50
pitch = 1

#### Option 1: Lattice

In [ ]:
lattice = openmc.RectLattice()
lattice.lower_left = (-N*pitch/2, -N*pitch/2)
lattice.pitch = (pitch, pitch)
lattice.universes = [[univ] * N] * N
print(lattice.universes.shape)

In [ ]:
bound = openmc.model.RectangularPrism(N*pitch, N*pitch, boundary_type='reflective')
root_universe1 = openmc.Universe()
root_universe1.add_cell(openmc.Cell(region=-bound, fill=lattice))
root_universe1.plot(width=(N*pitch,N*pitch), color_by='material', pixels=(500,500))

In [ ]:
model.geometry = openmc.Geometry(root_universe1)

In [ ]:
statepoint = model.run()

### Option 2: "Flat" Geometry

Let's build the same geometry, but now without using a lattice.

In [ ]:
import numpy as np
xplanes = [openmc.XPlane(x0=x) for x in np.linspace(-N*pitch/2, N*pitch/2, N+1)]
yplanes = [openmc.YPlane(y0=y) for y in np.linspace(-N*pitch/2, N*pitch/2, N+1)]

xplanes[0].boundary_type = 'reflective'
xplanes[-1].boundary_type = 'reflective'
yplanes[0].boundary_type = 'reflective'
yplanes[-1].boundary_type = 'reflective'

In [ ]:
cells = []
for x in range(len(xplanes) - 1):
    for y in range(len(yplanes) - 1):
        left = xplanes[x]
        right = xplanes[x + 1]
        bottom = yplanes[y]
        top = yplanes[y + 1]
        
        x0 = (xplanes[x].x0 + xplanes[x + 1].x0) / 2
        y0 = (yplanes[y].y0 + yplanes[y + 1].y0) / 2
        
        cylinder = openmc.ZCylinder(r=0.4, x0=x0, y0=y0)
        cells.append(openmc.Cell(region=-cylinder, fill=mat1))
        cells.append(openmc.Cell(region=+cylinder & +left & -right & +bottom & -top, fill=mat2))

In [ ]:
root_universe2 = openmc.Universe(cells=cells)
root_universe2.plot(width=(N*pitch,N*pitch), pixels=(500,500))
root_universe2.plot(width=(N*pitch,N*pitch), color_by='material', pixels=(500,500))

We have created an identical geometry! Of course, this took more human effort to establish instead of using a lattice. But that's not the only important difference -- this "flat" version of the geometry will also be *slower*.

In [ ]:
model.geometry = openmc.Geometry(root_universe2)
statepoint = model.run()

This model ran about 3x slower than a (physically identical) version using a lattice. If you can build your geometry to take advantage of lattices, it is a good idea to do so.

# Using OpenMC Correctly

Monte Carlo "run strategy" generally consists of three choices:

- Number of batches (`model.settings.batches`)
- Number of inactive batches (`model.settings.inactive`)
- Number of particles per batch (`model.settings.particles`)
- Number of generations per batch (`model.settings.generations_per_batch`)

In addition, for k-eigenvalue calculations you elect the initial source distribution. We will individually explore each of these choices. First, let's describe what these terms mean exactly.

### What is a generation?

During each generation in OpenMC:

- `model.settings.particles` sites are sampled from a bank of fission sites
- Those particles are transported from birth until "death" (leakage or absorption)
- Any new neutrons which are produced during their lifetime (e.g., from fission) have their birth position, energy, and angle stored in a new fission bank.

Therefore, each time a new generation begins, the sites used to sample the neutron positions are those which were "banked" from the previous generation.

### What is a batch?

A batch is a group of generations as one statistical realization. By default, `model.settings.generations_per_batch` is unity, in which case a batch is synonymous with a generation.

### What is an inactive batch?

An inactive batch involves the transport of neutrons, but not accumulation of any tallies.

### What is an active batch?

An active batch involves the transport of neutrons AND accumulation of tallies.

## Inactive batches and initial source

For the very first generation run in OpenMC (the first generation of the first batch), we don't yet have a bank of sites to sample neutrons from. Therefore, the user has to provide a "guess" for the starting source, using `model.settings.source`. By default, this source distribution is taken to be a point source at the origin. In all k-eigenvalue cases, you don't know the source distribution, so whatever choice you make is a GUESS. However, we don't want to start accumulating tally statistics until our guessed source distribution undergoes enough generations so that it has statistically converged. Otherwise, even if you run a high number of particles/batch, your answers will be WRONG (but PRECISE!).

The factors you choose are:

- The starting source distribution. Ideally, you'd like this to be as close as possible to the true source distribution.
- The number of inactive batches. The higher this number is, the more likely you are to have evolved from your (WRONG) initial source to the true initial source.

To explore these effects, let's choose an initial source distribution which is clearly not the correct, converged source distribution. When choosing a source distribution, we will use the [`openmc.stats` module](https://docs.openmc.org/en/stable/pythonapi/stats.html), which contains probability distributions to sample from which we can use for energy, angle, and space.

In [ ]:
model = openmc.examples.pwr_pin_cell()
model.geometry.root_universe.plot()

In [ ]:
model.geometry.bounding_box

In [ ]:
bottom = openmc.ZPlane(z0=-150, boundary_type='vacuum')
top = openmc.ZPlane(z0=150, boundary_type='vacuum')
box = openmc.model.RectangularPrism(0.63*2, 0.63*2, boundary_type='reflective')
outer_cell = openmc.Cell(region=-box & +bottom & -top, fill=model.geometry.root_universe)
root_universe = openmc.Universe(cells=[outer_cell])
model.geometry.root_universe = root_universe

Let's make a source distribution which only exists over the lower half of the pincell -- as if the entire upper half had no fission at all. Clearly this is a very bad guess!

In [ ]:
x = openmc.stats.Uniform(-0.63, 0.63)
y = openmc.stats.Uniform(-0.63, 0.63)
z = openmc.stats.Uniform(-150, 0)
space = openmc.stats.CartesianIndependent(x=x, y=y, z=z)
model.settings.source = openmc.IndependentSource(space=space)

In order to "see" the effect of having a wrong initial source distribution, we will need to add a tally to visualize a quantity affected by a wrong source distribution. Let's take a look at the heating distribution.

In [ ]:
heat = openmc.Tally()
heat.scores = ['kappa-fission']

mesh = openmc.RegularMesh()
mesh.lower_left = (-0.63, -0.63, -150)
mesh.upper_right = (0.63, 0.63, 150)
mesh.dimension = (1, 1, 50)

heat.filters = [openmc.MeshFilter(mesh=mesh)]

model.tallies = [heat]

Now, we will run this model with different choices for numbers of inactive batches. As long as we run enough inactive batches, we can evolve away from this very wrong source distribution to the true, cosine distribution we expect.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
model.settings.particles = 1000 

fig, ax = plt.subplots()

for i in [10, 20, 100, 500]:
    model.settings.batches = i + 100
    statepoint = model.run(output=False, apply_tally_results=True)    
    ax.plot(heat.get_values().flatten(), label='{} inactive'.format(i))
    
plt.legend()
plt.grid()
plt.xlabel('Height')
plt.ylabel('Kappa-Fission (unnormalized)')

We can see that we need more and more inactive batches if we want our fission source distribution to approach the true distribution. If you increase the number of particles, the statistical noisiness will decrease, but it won't help us to get closer to the true distribution for the sampled source. As a rule of thumb on selecting the inactive batches:

- Small problems (critical assemblies, pin cell): O(10) batches
- Medium problems (assemblies, fast reactor): O(10)–O(100) batches
- Large problems (PWR core, spent fuel pool): O(100)–O(1000) batches?

The dominance ratio, or the ratio of the second-largest eigenvalue to the largest eigenvalue, dictates the convergence rate of power iteration (each generation in a Monte Carlo simulation is a power iteration).

If we change our initial guess for the source distribution to something more realistic, we'll be able to use fewer inactive batches before we start collecting tally statistics.

In [ ]:
z = openmc.stats.Uniform(-150, 150)
space = openmc.stats.CartesianIndependent(x=x, y=y, z=z)
model.settings.source = openmc.IndependentSource(space=space)

In [ ]:
model.settings.particles = 1000

fig, ax = plt.subplots()

for i in [10, 20, 100, 500]:
    model.settings.batches = i + 100
    statepoint = model.run(output=False, apply_tally_results=True)
    ax.plot(heat.get_values().flatten(), label='{} inactive'.format(i))
    
plt.legend()
plt.grid()
plt.xlabel('Height')
plt.ylabel('Kappa-Fission (unnormalized)')

## Shannon Entropy

Shannon entropy is a concept from information theory that characterizes how much "information" a bit stream stores. In the context of eigenvalue calculations, it has been shown that when a source distibution is discretized over a mesh, the entropy of the source probability converges as the distribution itself reaches stationarity. Shannon entropy is defined as:

$$ H = - \sum_i p_i \log_2 p_i $$

where $p_i$ is the fraction of source particles in mesh cell $i$. In essence, we superimpose a mesh onto our geometry, and assess stationarity of the fission source by looking for stationarity in the Shannon entropy.

We'll need to create a `RegularMesh` object that will be used to count source particles over and calculate Shannon entropy. We can use the same mesh used for talling the fission source earlier (though this is not a requirement).

In [ ]:
model.settings.entropy_mesh = mesh

In [ ]:
statepoint = model.run()

In [ ]:
with openmc.StatePoint(statepoint) as sp:
    entropy = sp.entropy
    
plt.plot(entropy)
plt.xlabel('Batch')
plt.ylabel('Shannon entropy')

You should choose your number of inactive batches once the Shannon entropy plateaus to a constant value. From this example, we are not using enough particles to see this trend, because our statistical noise is quite high.

In [ ]:
model.settings.particles = 10000
statepoint = model.run(output=False)
with openmc.StatePoint(statepoint) as sp:
    entropy = sp.entropy
    
plt.plot(entropy)
plt.xlabel('Batch')
plt.ylabel('Shannon entropy')

From the above, it looks like we can justify using about 200 inactive batches. It is common to use a moving window average in order to make this determination more quantitatively.

Note that you will not want to run with Shannon entropy enabled all the time, because the simulation will be quite a bit slower.

## Active batches

The number of active batches (`settings.batches - settings.inactive`) are the number of tally realizations we use to accumulate tally statistics. The higher this number, the lower the statistical noise in our result (but remember that unless we properly chose the inactive batches, we could be PRECISE but WRONG!).

The standard deviation in a Monte Carlo tally obeys the central limit theorem, assuming that each batch of neutrons is independent and identically distributed from other batches (more on this later). Then, 

$$ \sigma\propto\frac{1}{\sqrt{N}}$$

where $N$ is the number of tally realizations. In other words, if we want to decrease the standard deviation in our simulation by a factor of 2, we would need to run 4 times as many particles.

In [ ]:
# re-init the settings in order to remove the Shannon entropy mesh
model.settings = openmc.Settings()

space = openmc.stats.CartesianIndependent(x=x, y=y, z=z)
model.settings.source = openmc.IndependentSource(space=space)

model.settings.inactive = 100
model.settings.batches = 200
model.settings.particles = 500
statepoint = model.run(apply_tally_results=True)

### Tally triggers

You most likely will not know, the first time you run your model, what the statistical error will be in your tallies of interest (this will depend on how many "scores" occur to each tally). However, OpenMC has the ability to  keep running batches until a certain condition on a tally is met. These conditions can be set using the variance, standard deviation, or relative error.

In [ ]:
tally_values = heat.get_values(value='rel_err').flatten()
print(tally_values.max())

Let's now have OpenMC keep adding batches until reaching a maximum relative error of 10% in the heating tally. With triggers, we need to tell OpenMC how frequently to check whether the triggers are satisfied (`batch_interval`) and a maximum number of batches to run.

In [ ]:
trigger = openmc.Trigger('rel_err', 0.10)
heat.triggers = [trigger]
model.tallies = [heat]

In [ ]:
model.settings.trigger_active = True
model.settings.trigger_batch_interval = 5
model.settings.trigger_max_batches = 1000

In [ ]:
statepoint = model.run(apply_tally_results=True)

In [ ]:
tally_values = heat.get_values(value='rel_err').flatten()
print(tally_values.max())

We can also add a trigger to the multiplication factor. When you have multiple triggers, the simulation will only terminate once all are satisfied (or you reach `max_batches`).

In [ ]:
model.settings.keff_trigger = {'type': 'std_dev', 'threshold': 0.00200}

In [ ]:
statepoint = model.run(apply_tally_results=True)

## Inter-Cycle Correlation

For the central limit theorm to apply, each batch of particles which is used to assembly a tally realization should be INDEPENDENT. However, this is obviously not the case, since we sample the birth sites for neutrons in batch $i$ from the the fission sites from batch $i-1$. This is called "inter-cycle correlation," and means that the actual variances reported by a Monte Carlo code *underpredict* the true variance.

One way that you can increase the independence of batches is to set `settings.generations_per_batch` to a number greater than 1. This will effectively increase the independence of each batch by reduce the cross-correlation between the fission bank of other batches.

You can quantify the extent of inter-cycle correlation by running OpenMC repeated times using different seeds. The mean and standard deviation of those estimates have the inter-cycle correlation removed. In order for these runs to be faster, we can drastically reduce the number of inactive batches (of course, our answers are wrong, but right now we are only exploring the inter-cycle correlation aspect, which is independent of the inactive batch choice).

In [ ]:
model.tallies = []

model.settings.trigger_active = False
model.settings.inactive = 10
model.settings.batches = 110
model.settings.particles = 4000

In [ ]:
n = 1

k_values = []
k_std_dev_values = []
while n <= 5:
    model.settings.seed = n
    statepoint = model.run(output=False)
    
    with openmc.StatePoint(statepoint) as sp:
        k_values.append(sp.keff.nominal_value)
        k_std_dev_values.append(sp.keff.std_dev)
        
    print('Ran OpenMC with seed = {0} ... k = {1} +/- {2}'.format(n, k_values[-1], k_std_dev_values[-1]))
    
    n += 1

In [ ]:
import numpy as np
mean = np.mean(k_values)
std_dev = np.std(k_values)

print(mean, std_dev)

Notice how the standard deviation we compute of the multiple independent runs is HIGHER than the values reported from any individual OpenMC solve. This occurs because each batch is not truly independent of the other batches.

In [ ]:
model.settings.generations_per_batch = 2

In [ ]:
n = 1

k_values = []
k_std_dev_values = []
while n <= 5:
    model.settings.seed = n
    statepoint = model.run(output=False)
    with openmc.StatePoint(statepoint) as sp:
        k_values.append(sp.keff.nominal_value)
        k_std_dev_values.append(sp.keff.std_dev)
    print('Running OpenMC with seed = {0} ... k = {1} +/- {2}'.format(n, k_values[-1], k_std_dev_values[-1]))
    
    n += 1

In [ ]:
mean = np.mean(k_values)
std_dev = np.std(k_values)

print(mean, std_dev)

Note how our sample standard deviation taken from multiple independent OpenMC runs is now closer to the value estimated from a single OpenMC run.